In [100]:
import csv
import logging
from datetime import datetime
from typing import List

import dask.distributed
import dask.utils
import numpy as np
import planetary_computer as pc
import xarray as xr

from IPython.display import display
from pystac_client import Client

from shapely.geometry import Polygon
from odc.stac import configure_rio, stac_load

# import icecream as ic
import geopandas as gpd
import folium

In [123]:
class Sentinel2DataFetcher:
    def __init__(self):

        self.client = dask.distributed.Client()
        configure_rio(cloud_defaults=True, client=self.client)
        display(self.client)

        # self.plancomp_catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
        self.aws_catalog = Client.open("https://earth-search.aws.element84.com/v1/")


    def search_sentinel2(self, start_date: str, end_date: str, aoi) -> List[dict]:
        """
        Search for Sentinel-2 data within a time period and area of interest.

        Parameters:
        - start_date (str): Start date in the format "YYYY-MM-DD".
        - end_date (str): End date in the format "YYYY-MM-DD".
        - aoi (List[Tuple[float, float]]): Area of interest as a list of (longitude, latitude) tuples.

        Returns:
        - A list of items (dicts) matching the search criteria.
        """
        try:
            # Validate date format
            datetime.strptime(start_date, "%Y-%m-%d")
            datetime.strptime(end_date, "%Y-%m-%d")

            # Validate date range
            if start_date > end_date:
                raise ValueError("Start date must be before end date.")

            # Create a polygon from the area of interest
            geometry = aoi

            # Search for data in the Sentinel-2 collection
            query = self.aws_catalog.search(
                collections=['sentinel-2-l2a'],
                datetime=f"{start_date}/{end_date}",
                bbox=geometry
                # limit=10  # Adjust limit as necessary
            )

            # Extract the STAC items and return them
            return query  # [item.to_dict() for item in items]

        except Exception as e:
            logging.error(f"An error occurred: {e}")
            return []  # Return an empty list or handle as needed


    def parse_polygon(self, polygon_str):
        # Remove 'POLYGON ((' and '))', then split by ', '
        coordinates = polygon_str[10:-2].split(', ')
        # Convert string coordinates to tuples of floats
        return [(float(lon), float(lat)) for lon, lat in (point.split(' ') for point in coordinates)]


    def get_polygon_extent(self, polygon_coordinates):
        lons, lats = zip(*polygon_coordinates)
        return [min(lons), min(lats), max(lons), max(lats)]


    def get_extend_from_aoi_code(self, file_path, experiment_code):
        with open(file_path, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row['Experiment_Code'] == experiment_code:
                    polygon_str = row['Field_Polygon']
                    polygon_coordinates = self.parse_polygon(polygon_str)

                    polygon_extent = list(self.get_polygon_extent(polygon_coordinates))
                    return polygon_extent
        return None


In [124]:
# Establish connection to Sentinel2DataFetcher
s2_fetcher = Sentinel2DataFetcher()

/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42871 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:42871/status,
Dashboard: http://127.0.0.1:42871/status,Workers: 4
Total threads: 8,Total memory: 15.44 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45055,Workers: 4
Dashboard: http://127.0.0.1:42871/status,Total threads: 8
Started: Just now,Total memory: 15.44 GiB
Comm: tcp://127.0.0.1:43109,Total threads: 2
Dashboard: http://127.0.0.1:41721/status,Memory: 3.86 GiB
Nanny: tcp://127.0.0.1:34551,


In [128]:
# Example usage
file_path = '/home/hn/Garden/eo_playground/data/g2f_2017_field_polygon.csv'  # Replace with your actual file path
aoi_code = 'ARH1'  # Replace with the desired experiment code
aoi_extent = s2_fetcher.get_extend_from_aoi_code(file_path, aoi_code)
print(aoi_extent)

[-90.76089, 34.720138, -90.759768, 34.729874]


In [129]:
# Define the time period
start_date = '2023-10-01'
end_date = '2023-10-31'

# Perform the search
sentinel2_query = s2_fetcher.search_sentinel2(start_date, end_date, aoi_extent)
print(sentinel2_query)

In [130]:
# Fetch the items
s2_items = list(sentinel2_query.items())
print(s2_items)
print(f"Found: {len(s2_items):d} datasets")

[<Item id=S2A_15SXU_20231027_0_L2A>, <Item id=S2A_15SYU_20231027_0_L2A>, <Item id=S2B_15SXU_20231022_0_L2A>, <Item id=S2B_15SYU_20231022_0_L2A>, <Item id=S2A_15SXU_20231017_0_L2A>, <Item id=S2A_15SYU_20231017_0_L2A>, <Item id=S2B_15SXU_20231012_0_L2A>, <Item id=S2B_15SYU_20231012_0_L2A>, <Item id=S2A_15SXU_20231007_0_L2A>, <Item id=S2A_15SYU_20231007_0_L2A>, <Item id=S2B_15SXU_20231002_0_L2A>, <Item id=S2B_15SYU_20231002_0_L2A>]
Found: 12 datasets


In [131]:
cfg = {
    "sentinel-2-l2a": {
        "assets": {
            "*": {"data_type": "uint16", "nodata": 0},
            "SCL": {"data_type": "uint8", "nodata": 0},
            "visual": {"data_type": "uint8", "nodata": 0},
        },
    },
    "*": {"warnings": "ignore"},
}

In [132]:
resolution = 10
SHRINK = 4
if s2_fetcher.client.cluster.workers[0].memory_limit < dask.utils.parse_bytes("4G"):
    SHRINK = 8  # running on Binder with 2Gb RAM

if SHRINK > 1:
    resolution = resolution * SHRINK

ds = stac_load(
    s2_items,
    chunks={"x": 2048, "y": 2048},
    stac_cfg=cfg,
    patch_url=pc.sign,
    resolution=resolution,
)

print(f"Bands: {','.join(list(ds.data_vars))}")
display(ds)

ds = stac_load(
    s2_items,
    bands=["red", "green", "blue", "nir", "scl"],
    resolution=resolution,
    chunks={"x": 2048, "y": 2048},
    stac_cfg=cfg,
    patch_url=pc.sign,
)

print(f"Bands: {','.join(list(ds.data_vars))}")
display(ds)

/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/distributed/worker_memory.py:504: FutureWarning: The `Nanny.memory_limit` attribute has been moved to `Nanny.memory_manager.memory_limit
  warnings.warn(


Bands: aot,blue,coastal,green,nir,nir08,nir09,red,rededge1,rededge2,rededge3,scl,swir16,swir22,visual,wvp,aot-jp2,blue-jp2,coastal-jp2,green-jp2,nir-jp2,nir08-jp2,nir09-jp2,red-jp2,rededge1-jp2,rededge2-jp2,rededge3-jp2,scl-jp2,swir16-jp2,swir22-jp2,visual-jp2,wvp-jp2


<xarray.Dataset>
Dimensions:       (y: 2745, x: 5244, time: 12)
Coordinates:
  * y             (y) float64 3.9e+06 3.9e+06 3.9e+06 ... 3.79e+06 3.79e+06
  * x             (x) float64 6e+05 6.001e+05 6.001e+05 ... 8.097e+05 8.097e+05
    spatial_ref   int32 32615
  * time          (time) datetime64[ns] 2023-10-02T16:54:01.804000 ... 2023-1...
Data variables: (12/32)
    aot           (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue          (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    coastal       (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green         (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir           (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir08         (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    ...            ...
    rededge3-jp2  (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl-jp2       (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir16-jp2    (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir22-jp2    (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    visual-jp2    (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    wvp-jp2       (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

Bands: red,green,blue,nir,scl


<xarray.Dataset>
Dimensions:      (y: 2745, x: 5244, time: 12)
Coordinates:
  * y            (y) float64 3.9e+06 3.9e+06 3.9e+06 ... 3.79e+06 3.79e+06
  * x            (x) float64 6e+05 6.001e+05 6.001e+05 ... 8.097e+05 8.097e+05
    spatial_ref  int32 32615
  * time         (time) datetime64[ns] 2023-10-02T16:54:01.804000 ... 2023-10...
Data variables:
    red          (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

In [133]:
def to_float(ds):
    _ds = ds.astype("float32")
    nodata = ds.attrs.pop("nodata", None)
    if nodata is None:
        return _ds
    return _ds.where(ds != nodata)

# like .astype(float32) but taking care of nodata->NaN mapping
nir = to_float(ds.nir)
red = to_float(ds.red)


def calculate_ndvi(ds):
    # Calculate NDVI
    ndvi = (nir - red) / (nir + red)
    return ndvi

# Your existing code to load the dataset
# ...

# Calculate NDVI for each image
ndvi_ds = calculate_ndvi(ds)
# ndvi_ds = ndvi_ds.fillna(0)

# Compute mean NDVI across all images
mean_ndvi = ndvi_ds.mean(dim=['time', 'x', 'y'])

# Handle NaN values if present
mean_ndvi = mean_ndvi.fillna(0)

print(f"Mean NDVI: {mean_ndvi.values}")

/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


Mean NDVI: 0.29083263874053955


In [134]:
# Unpack the extent into individual coordinates
min_lon, min_lat, max_lon, max_lat = aoi_extent

# Initialize a Folium map at the center of your extent
map_center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]
m = folium.Map(
    location=map_center, 
    zoom_start=15, 
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri', name = 'Esri Satellite',)

# Add a rectangle to the map
folium.Rectangle(
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],
    color="#ff7800",
    fill=True,
    fill_color="#ffff00",
    fill_opacity=0.2,
    tooltip=f'Mean NDVI: {mean_ndvi.values:.3f}',
).add_to(m)

/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


In [135]:
import matplotlib.pyplot as plt

# # Normalize NDVI data
normalized_ndvi = (mean_ndvi - mean_ndvi.min()) / (mean_ndvi.max() - mean_ndvi.min())

# Convert the NDVI data to an RGBA image using a colormap
colormap = plt.get_cmap('viridis')  # You can choose a different colormap if you prefer
ndvi_image = colormap(normalized_ndvi.values, bytes=True)  # Convert to an 8-bit RGBA image

# Flip the image vertically for correct orientation
ndvi_image = np.flipud(ndvi_image)

# Add the NDVI overlay to the map
folium.raster_layers.ImageOverlay(
    image=ndvi_image,
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],
    colormap='viridis',  # This should match the colormap used earlier
).add_to(m)


In [136]:
m

In [110]:
# Generate a grid of points within the bounds of your NDVI data
# This step depends on the resolution and extent of your NDVI data
latitudes = np.linspace(min_lat, max_lat, num=10)  # Adjust 'num' as needed
longitudes = np.linspace(min_lon, max_lon, num=10)  # Adjust 'num' as needed

# Create invisible markers with tooltips
for lat in latitudes:
    for lon in longitudes:
        folium.Marker(
            [lat, lon],
            icon=folium.Icon(icon_color='rgba(0,0,0,0)'),  # Invisible icon
            tooltip=f'Mean NDVI: {mean_ndvi.values:.3f}'  # Adjust formatting as needed
        ).add_to(m)

# Save and display the map
# m.save('ndvi_map_with_tooltips.html')

/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
Task exception was never retrieved
future: <Task finished name='Task-2814220' coro=<Client._gather.<locals>.wait() done, defined at /home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/distributed/client.py:2196> exception=AllExit()>
Traceback (most recent call last):
  File "/home/hn/miniconda3/envs/eoenv/lib/python3.10/site-packages/distributed/client.py", line 2205, in wait
    

KeyboardInterrupt: 

In [ ]:
m